# Logging of the project

In this notebook you will find all the steps it took us to acurately predict the temperature of nuclear waste canister. 

### **Imports** 

In [44]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer

### **Loading of the datasets** 

In [45]:
coordinates_test = pd.read_csv("data/Coordinates_Test.csv")
coordinates_train = pd.read_csv("data/Coordinates_Training.csv")
humidity_test = pd.read_csv("data/Test_Time_humidity.csv")
pressure_test = pd.read_csv("data/Test_Time_pressure.csv")
humidity_train = pd.read_csv("data/Training_data_humidity.csv")
pressure_train = pd.read_csv("data/Training_data_pressure.csv")
temperature_train = pd.read_csv("data/Training_data_temperature.csv")

### **Visualizing the datasets**

In [46]:
display(coordinates_train.sample(10))
coordinates_train.info()
coordinates_train["Material"].unique()

,Unnamed: 0,Sensor ID,Index,Material,Coor X [m],Coor Y [m],Coor Z [m],R [m]
201,201,N_202,202,OPA,-5.201460,15.299153,12.403412,13.449900
872,872,N_873,873,OPA,-3.130858,29.361992,0.767665,3.223598
502,502,N_503,503,OPA,-3.540187,30.940394,0.663370,3.601803
323,323,N_324,324,OPA,13.143439,5.966490,9.637879,16.298426
455,455,N_456,456,OPA,11.116220,20.453765,-10.337113,15.179798
203,203,N_204,204,OPA,12.693046,6.105267,-8.705361,15.391450
418,418,N_419,419,OPA,6.526778,22.361181,17.981810,19.129672
769,769,N_770,770,OPA,-0.817869,16.239954,2.779201,2.897045
541,541,N_542,542,OPA,0.648558,21.695949,-2.129974,2.226525
880,880,N_881,881,OPA,1.231808,38.666481,-2.641580,2.914669


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  900 non-null    int64  
 1   Sensor ID   900 non-null    object 
 2   Index       900 non-null    int64  
 3   Material    900 non-null    object 
 4   Coor X [m]  900 non-null    float64
 5   Coor Y [m]  900 non-null    float64
 6   Coor Z [m]  900 non-null    float64
 7   R [m]       900 non-null    float64
dtypes: float64(4), int64(2), object(2)
memory usage: 56.4+ KB


array(['OPA', 'SHCR', 'GBM', 'EDZ', 'VOID', 'CAN', 'BBLOCK'], dtype=object)

We can see there are no missing data and apparently no false measures (outliers) on the position, but some columns are useless like the numbering of rows, the Sensor ID, and the index. Because the rows are in an ascending order and as long as the indices match between the files, the name of the sensor and its number doesn't matter. We are also renaming the columns to make it easier later.

In [47]:
coordinates_train = coordinates_train[["Material", "Coor X [m]", "Coor Y [m]", "Coor Z [m]", "R [m]"]].copy()
coordinates_test = coordinates_test[["Material", "Coor X [m]", "Coor Y [m]", "Coor Z [m]", "R [m]"]].copy()

# changing the column names for faster typing later
new_col_names: dict = {
    "Coor X [m]": "x",
    "Coor Y [m]": "y",
    "Coor Z [m]": "z",
    "R [m]": "r"
}
coordinates_train.rename(columns = new_col_names, inplace=True)
coordinates_test.rename(columns = new_col_names, inplace=True)

display(coordinates_train.head(5))

,Material,x,y,z,r
0,OPA,0.208042,14.436936,-2.875503,2.883019
1,OPA,-8.970832,28.229841,-0.134437,8.971839
2,OPA,-14.289501,6.685726,-10.399048,17.672862
3,OPA,6.114855,2.685645,-3.189981,6.896914
4,OPA,4.048845,48.708590,11.260503,11.966289


In [48]:
display(humidity_train.iloc[:,100:130])
mean_humidity = humidity_train.mean(axis=0).iloc[1:] ## not keeping the time for the mean
mean_humidity.dropna(inplace=True)
print(f"Global mean: {np.mean(mean_humidity, axis=0)}, variance: {np.std(mean_humidity, axis=0)}")

,N_100,N_101,N_102,N_103,N_104,N_105,N_106,N_107,N_108,N_109,...,N_120,N_121,N_122,N_123,N_124,N_125,N_126,N_127,N_128,N_129
0,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
1,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
2,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
3,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
4,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
5,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
6,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
7,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
8,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
9,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100


Global mean: 98.19472180013585, variance: 5.56673083885869


The humidity doesn't seem to contain much information and we will discard it for our first model.

In [49]:
display(pressure_train.sample(10))
pressure_train.isnull().sum().sum() # See how many missing values there are

,M.Time[d],N_1,N_2,N_3,N_4,N_5,N_6,N_7,N_8,N_9,...,N_891,N_892,N_893,N_894,N_895,N_896,N_897,N_898,N_899,N_900
10,1606,261.820107,1447.726782,1654.833464,1313.827074,1472.228901,1479.439225,1647.804197,1507.797530,1391.036271,...,-56725.22725,316.763658,512.436753,682.533003,-226.697799,-60269.58734,-773.110572,418.949689,-62252.85078,905.510398
31,7028,-247.388122,1281.580935,1559.693354,984.004120,1567.306355,1559.751146,1470.519975,1549.189461,1197.920901,...,-29736.69634,-228.482724,61.758113,261.733048,-946.253126,-48070.66786,-1508.850859,-203.434323,-57072.76790,588.075131
24,2871,220.590628,1764.328854,1627.127567,1177.595176,1514.708687,1502.646936,1583.963759,1599.758946,1421.908767,...,-45129.22197,362.488181,660.676556,919.286100,-322.670460,-56816.59191,-935.809451,419.367328,-60670.56052,1160.232987
16,1770,313.048071,1489.976923,1648.304503,1285.272217,1466.829385,1476.779438,1635.385041,1496.376810,1376.967077,...,-54801.95550,567.604919,816.582215,993.602913,-40.453090,-59766.39804,-734.129972,628.027391,-61841.31880,1173.314152
6,1572,270.952332,1457.058588,1655.697105,1319.591329,1473.223592,1480.020813,1649.839730,1510.319456,1395.550471,...,-57191.21953,213.937626,408.653531,645.271850,-372.715269,-60375.63042,-777.774434,339.314345,-62585.29747,837.420404
29,5587,-126.443948,1427.415600,1599.832439,1044.165672,1582.561190,1568.662633,1518.937457,1609.704476,1287.358849,...,-32874.04125,-92.627052,206.471809,410.454022,-794.210744,-50782.56296,-1354.899393,-47.678273,-58459.67072,736.076259
26,3616,106.274939,1684.808222,1628.652425,1137.270914,1555.532477,1534.928239,1571.155251,1643.038450,1405.974281,...,-40500.99800,195.130120,500.025021,731.597320,-495.292741,-55013.86874,-1075.597360,259.967716,-60047.50393,1023.587342
20,2090,327.870118,1670.042287,1635.025242,1239.640727,1468.399228,1476.239040,1611.330120,1509.888862,1390.031486,...,-51569.22808,561.512961,843.901837,1110.664107,-96.492185,-58863.88557,-772.727303,605.028927,-61440.53123,1263.655853
0,1554,281.143955,1462.382827,1656.041967,1322.584393,1473.656845,1480.325846,1650.794639,1511.445756,1397.850463,...,-57423.47996,202.296319,395.883971,651.026983,-405.358355,-60428.68990,-768.389510,336.655683,-63100.53626,831.154054
13,1662,276.089676,1445.439966,1652.937807,1304.070726,1470.281668,1478.459543,1643.928649,1503.114566,1384.002387,...,-56020.85926,460.869740,674.765620,808.421271,-96.448798,-60092.11262,-751.045993,541.729133,-62043.75043,1033.392931


96

We see there are 96 missing values for the pressure, we'll use KNN imputation to replace them, with 2 neighbors to hopefully make the data coherent in its progression.

In [50]:
imputer = KNNImputer(missing_values = np.nan, n_neighbors = 2).set_output(transform="pandas")
pressure_train = imputer.fit_transform(pressure_train)
pressure_test = imputer.fit_transform(pressure_test)
pressure_train.isnull().sum().sum() 


0

In [51]:
display(temperature_train.head(10))

,M.Time[d],N_1,N_2,N_3,N_4,N_5,N_6,N_7,N_8,N_9,...,N_891,N_892,N_893,N_894,N_895,N_896,N_897,N_898,N_899,N_900
0,1554,17.623059,17.154220,17.641578,17.455701,NaN,16.604935,17.662407,16.503001,16.943823,...,17.503931,17.225297,17.498277,17.268529,17.573474,17.412215,17.526257,17.364940,24.026562,17.538194
1,1556,17.620860,17.154263,17.641672,17.455850,16.415312,16.605042,17.662519,16.503121,16.943985,...,17.510776,17.223290,17.498581,17.267488,17.578925,17.409841,17.522860,17.363663,33.729552,17.537460
2,1558,17.618608,17.154303,17.641766,17.455998,16.415377,16.605148,17.662632,16.503240,16.944146,...,17.534085,17.223733,17.501874,2872.837827,17.599256,17.407913,17.520157,17.363850,41.602481,17.537433
3,1560,17.616334,2717.706176,17.641859,17.456146,16.415440,16.605254,17.662744,16.503357,16.944307,...,17.581610,17.228355,17.509670,17.266326,17.640317,17.406770,17.518750,17.366504,48.218980,17.538652
4,1563,17.612991,17.154388,17.642000,17.456367,16.415531,16.605414,17.662912,16.503530,16.944544,...,17.723547,17.249726,17.535358,17.267759,17.757592,17.408069,17.521699,17.379102,56.258743,17.545154
5,1567,17.609008,17.154454,17.642187,17.456661,16.415646,16.605626,17.663136,16.503753,16.944855,...,2766.426947,17.310388,17.598978,17.275449,18.010005,17.417672,17.538472,17.415806,64.775395,17.565561
6,1572,17.605614,17.154568,17.642420,17.457028,16.415779,16.605890,17.663415,16.504023,16.945233,...,18.634191,17.438013,17.724790,17.297646,18.444537,17.444923,17.583032,17.495517,73.147489,17.612783
7,1578,17.605318,17.154841,17.642700,17.457465,16.415925,16.606206,17.663749,16.504334,16.945670,...,19.530270,17.656072,17.934088,17.346024,19.076916,17.501008,17.670392,17.636533,80.996009,17.701098
8,1585,17.612025,17.155530,17.643025,17.457972,16.416077,16.606574,17.664138,16.504678,16.946154,...,20.726138,17.976554,18.239215,17.433777,19.889638,17.596132,17.812325,17.851145,88.108493,17.842181
9,1595,17.638763,17.157931,17.643489,17.458689,16.416256,16.607098,17.664689,16.505137,16.946813,...,22.524169,18.511739,18.750719,17.618164,21.073668,17.779369,18.073014,18.223945,95.570527,18.100318


But here we notice sudden spikes in the temperature going from about 20 to 2'000+ (the mineral would most likly melt at that temperature) and going back down to 20 a few days later. We can consider those value as measurement errors. We can replace those outliers by the mean of the 2 nearest neighbors if they are above a threshold of 500 (we can see it should work well by looking at the graph below)

In [52]:
# We need to be careful and not taking into account the first column with high values of time
columns = temperature_train.columns.difference(["M.Time[d]"])

## Commented out a it takes 15s to run
# sns.boxplot(temperature_train.iloc[:,1:])

temperature_train[columns] = temperature_train[columns].mask(temperature_train[columns] >= 500, np.nan)
# temperature_train.iloc[:, :10]

## We impute the missing data with the 2 nearest neighbors as done with the pressure
temperature_train = imputer.fit_transform(temperature_train)
temperature_train.isnull().sum().sum() # check that it worked

0

### **Feature engineering**